In [1]:
import auxiliary_tools
from tqdm import tqdm 
from geopy.distance import geodesic

In [2]:
import array
import random
import numpy as np
import json
import pickle
import numpy
from math import sqrt
from deap import algorithms
from deap import base
from deap import benchmarks
from deap.benchmarks.tools import diversity, convergence, hypervolume
from deap import creator
from deap import tools

In [3]:
def generate_individual(creator, route_requests, rand_dist_min, rand_dist_max):
    individual = []
    for request in route_requests:
        
        rand_distance = random.randint(rand_dist_min, rand_dist_max)/1000
        rand_angle = random.randint(1, 360)
        
        gene = geodesic(kilometers=rand_distance).destination(request, rand_angle)[:2]
        
        individual.append(gene)
    individual = np.array(individual)
    return creator.individual(individual)

In [4]:
def mutation(individual, mutation_probability, rand_dist_min, rand_dist_max):
    mutated_individual = []
    for gene in individual:
        if random.random() < mutation_probability:
            rand_distance = random.randint(rand_dist_min, rand_dist_max)/1000
            rand_angle = random.randint(1, 360)
            
            mutated_gene = geodesic(kilometers=rand_distance).destination(gene, rand_angle)[:2]
            mutated_individual.append( mutated_gene )
        else:
            mutated_individual.append( gene )
    return  creator.individual(np.array(mutated_individual))

In [5]:
def crossover(individual_a, individual_b, crossover_probability):
    child_a = []
    child_b = []

    for i, (gene_a, gene_b) in enumerate(zip(individual_a, individual_b)):
        if random.random() < crossover_probability:
            child_a.append(gene_b)
            child_b.append(gene_a)
        else:
            child_a.append(gene_a)
            child_b.append(gene_b)

    return (creator.individual(np.array(child_a)), creator.individual(np.array(child_b)))

In [6]:
def client_fitness(route_requests, individual):
    c_fitness = []
    for i in range(len(route_requests)):
        request_r = route_requests[i]
        request_origin = [request_r[0], request_r[1]]
        vs_individual = individual[i]
        vs_destination = vs_individual
        c_fitness.append(auxiliary_tools.getGeoDistanceETA_OSRM(request_origin, vs_destination, 5005, 'walking'))
    fitness_value = np.sum([f[0] for f in c_fitness])
    return fitness_value

def operator_fitness(individual, penalty_const):
    ori_dest = [(first, second) for first, second in zip(individual, individual[1:])]
    penalty_sum = 0
    for pair in ori_dest:
        if max(pair[0] != pair[1]) == True:
            penalty_sum+=penalty_const
    o_fitness = []
    for od_r in ori_dest:
        o_fitness.append(auxiliary_tools.getGeoDistanceETA_OSRM(od_r[0], od_r[1], 5004, 'driving'))
        
    fitness_value = np.sum([f[0] for f in o_fitness]) + penalty_sum
    return fitness_value

def fitness(individual, route_requests, penalty_const):
    import time
#     start_time = time.time()
    from pexecute.thread import ThreadLoom
    loom = ThreadLoom(max_runner_cap=10)
    
    loom.add_function(client_fitness, [route_requests, individual], {})
    loom.add_function(operator_fitness, [individual, penalty_const], {})

    output = loom.execute()
    client_f = output[0]['output']
    operator_f = output[1]['output']
#     print("--- %s seconds ---" % round(time.time() - start_time))
    return client_f, operator_f

In [7]:
penalty_const = auxiliary_tools.getPenaltyConst(2)

route_requests = auxiliary_tools.loadPrep(2, 1)

crossover_probability = 0.4
mutation_probability = 0.5

rand_dist_min = 0
rand_dist_max = 500

population_size = 25
number_generations = 100

idx_evol = 5

# proposals:  123402
# requests 121427
# rides 46616


In [8]:
import time
start_time = time.time()

In [9]:
creator.create("min_fitness", base.Fitness, weights=(-1.0, -1.0))
creator.create("individual", list, fitness=creator.min_fitness)

toolbox = base.Toolbox()

toolbox.register("create_individual", generate_individual, creator, route_requests=route_requests,  rand_dist_min=rand_dist_min, rand_dist_max=rand_dist_max)
toolbox.register("initialize_population", tools.initRepeat, list, toolbox.create_individual)

toolbox.register("evaluate", fitness, route_requests=route_requests, penalty_const=penalty_const)
toolbox.register("crossover", crossover, crossover_probability=crossover_probability)
toolbox.register("mutate", mutation, mutation_probability=mutation_probability, rand_dist_min=rand_dist_min, rand_dist_max=rand_dist_min)

toolbox.register("select", tools.selNSGA2)

In [ ]:
data_ga = pickle.load(open("../../data_ga_evol_1.pkl", "rb"))